In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install tfa-nightly
!pip install import-ipynb
import import_ipynb
import tensorflow as tf
import numpy as np
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tqdm import tqdm
from fastprogress import master_bar, progress_bar
%cd '/content/drive/MyDrive/LMSLoss/paperLMSloss'

     |████████████████████████████████| 747kB 9.0MB/s 
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=62f67bae47626b02ac96943ee90891fb6f4561e272ecb9e53fe2b34a810a45d8
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
/content/drive/MyDrive/LMSLoss


/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking tensorflow_addons.layers.max_unpooling_2d.MaxUnpooling2D.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [ ]:
%run UnetModel.ipynb
# %run MumfordLoss.ipynb
%run augment_data.ipynb


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
swish (Swish)                   (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-10):
    '''Average dice coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    summation = K.sum(y_true + y_pred, axis=axes)
    return K.mean((2.0 * intersection + smooth) / (summation + smooth), axis=0)

def jaccard_coef(y_true, y_pred, smooth=1e-10):
    '''Average jaccard coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    union = K.sum(y_true + y_pred, axis=axes) - intersection
    return K.mean( (intersection + smooth) / (union + smooth), axis=0)

In [ ]:
x =  np.load("dataISIC2018/x.npy")
y = np.load("dataISIC2018/y.npy")
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state =42)

x_test_isic= x_test.astype(np.float32)
y_test_isic=y_test.astype(np.float32)

In [ ]:
x = np.asarray(np.load('./dataACDCA/x_crop_128.npy'),np.float32)
y = np.load('./dataACDCA/y_crop_128.npy')

In [ ]:
np.random.seed(123)
len_dev = x.shape[0]//5
len_train = x.shape[0] - len_dev
rand_index = np.random.permutation(x.shape[0])
x_test_acdc = x[rand_index[:len_dev]]
y_test_acdc = y[rand_index[:len_dev]]


In [ ]:
S_skin_bce  = seg_net((128,160,3),out_channels=1)
S_skin_dice  = seg_net((128,160,3),out_channels=1)
S_skin_lms = seg_net((128,160,3),out_channels=2)
S_skin_ms  = seg_net((128,160,3),out_channels=2)
S_skin_ac = seg_net((128,160,3),out_channels=2)

In [ ]:
S_acdc_bce_epi = seg_net(out_channels=1)
S_acdc_dice_epi = seg_net(out_channels=1)

S_acdc_bce = seg_net(out_channels=1)
S_acdc_dice = seg_net(out_channels=1)
S_acdc_lms = seg_net(out_channels=3)
S_acdc_ms = seg_net(out_channels=3)
S_acdc_ac = seg_net(out_channels=3)

In [ ]:
S_skin_bce.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightISIC_BCE/weight0.8554.h5')
S_skin_dice.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightISIC_diceloss/weight0.8779.h5')
S_skin_lms.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightISIC_LMS/ckpt_0.8890.h5')
S_skin_ms.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightISIC_MS/ckpt_0.8829.h5')

In [ ]:
S_acdc_bce_epi.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_BCE/weightepi0.9010.h5')
S_acdc_bce.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_BCE/weightendo0.8584.h5')
S_acdc_dice_epi.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_diceloss/weightepi0.9184.h5')
S_acdc_dice.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_diceloss/weightendo0.8844.h5')
S_acdc_lms.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_LMS/ckpt_0.9284.h5')
S_acdc_ms.load_weights('/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_MS/ckpt_0.9213.h5')
S_acdc_ac.load_weights("/content/drive/MyDrive/LMSLoss/paperLMSloss/weightACDCA_ac/ckpt_0.9112.h5")
S_skin_ac.load_weights("/content/drive/MyDrive/LMSLoss/paperLMSloss/weightISIC_ac/ckpt_0.8754.h5")

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
for i in range(y_test.shape[0]):
  y_pred = S_skin_bce(normalize(x_test_isic[i:i+1],y_test_isic[i:i+1]))
  y_pred2 = S_skin_dice(normalize(x_test_isic[i:i+1],y_test_isic[i:i+1]))
  y_pred3 = S_skin_lms(normalize(x_test_isic[i:i+1],y_test_isic[i:i+1]))
  y_pred4 = S_skin_ms(normalize(x_test_isic[i:i+1],y_test_isic[i:i+1]))
  y_pred5 = S_skin_ac(normalize(x_test_isic[i:i+1],y_test_isic[i:i+1]))
  #y_predShow = np.argmax(y_pred,axis = -1)
  #y_predShow2 = np.argmax(y_pred2,axis = -1)
  y_predShow = np.where(y_pred>=0.5,1.0,0.0)[...,0]
  y_predShow2 = np.where(y_pred2[...,0]>=0.5,1.0,0.0)
  y_predShow3 = np.argmax(y_pred3,axis = -1)
  y_predShow4 = np.argmax(y_pred4,axis = -1)
  y_predShow5 = np.argmax(y_pred5, axis=-1)
  plt.figure(i+1, figsize=(20,20))
  plt.subplot(171),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('image'), plt.axis('off')
  plt.subplot(172),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('groundtruth'),plt.contour(y_test[i,...,0], colors=['b'], levels=[0.0,1.0],linewidths=2), plt.axis('off')
  plt.subplot(173),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('BCE_loss'),plt.contour(y_predShow [0], colors=['b'], levels=[0.0,1.0],linewidths=2), plt.axis('off')
  plt.subplot(174),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('Dice_loss'),plt.contour(y_predShow2 [0], colors=['b'], levels=[0.0,1.0],linewidths=2), plt.axis('off')
  plt.subplot(175),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('Lac_loss'),plt.contour(y_predShow5 [0], colors=['b'], levels=[0.0,1.0],linewidths=2), plt.axis('off')
  plt.subplot(176),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('Lms_loss'),plt.contour(y_predShow4 [0], colors=['b'], levels=[0.0,1.0],linewidths=2), plt.axis('off')
  plt.subplot(177),plt.imshow(x_test[i,...].astype(np.int16)),plt.title('Our proposed'),plt.contour(y_predShow3 [0], colors=['b'], levels=[0.0,1.0],linewidths=2), plt.axis('off')
 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in range(x_test_acdc.shape[0]):
  y_pred = S_acdc_bce(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  y_pred_epi = S_acdc_bce_epi(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  y_pred2 = S_acdc_dice(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  y_pred2_epi = S_acdc_dice_epi(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  y_pred3 = S_acdc_lms(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  y_pred4 = S_acdc_ms(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  y_pred5 = S_acdc_ac(normalize(x_test_acdc[i:i+1],y_test_acdc[i:i+1]))
  #y_predShow = np.argmax(y_pred,axis = -1)
  #y_predShow2 = np.argmax(y_pred2,axis = -1)
  y_predShow = np.where(y_pred>=0.5,1.0,0.0)[...,0]
  y_predShow2 = np.where(y_pred2[...,0]>=0.5,1.0,0.0)
  y_predShow_epi = np.where(y_pred_epi>=0.5,1.0,0.0)[...,0]
  y_predShow2_epi = np.where(y_pred2_epi[...,0]>=0.5,1.0,0.0)
  y_predShow3 = np.argmax(y_pred3,axis = -1)
  y_predShow4 = np.argmax(y_pred4,axis = -1)
  y_predShow5 = np.argmax(y_pred5, axis=-1)
  plt.figure(i+1, figsize=(24,24))
  plt.subplot(171),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('image'), plt.axis('off')
  plt.subplot(172),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('groundtruth'),plt.contour(y_test_acdc[i,...,0], colors=['r','g'], levels=[0.0,1.0,2.0],linewidths=2.5), plt.axis('off')
  plt.subplot(173),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('BCE_loss'),plt.contour(y_predShow [0], colors=['g'], levels=[0.0,1.0],linewidths=2.5),plt.contour(y_predShow_epi [0], colors=['r'], levels=[0.0,1.0],linewidths=2.5) ,plt.axis('off')
  plt.subplot(174),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('Dice_loss'),plt.contour(y_predShow2 [0], colors=['g'], levels=[0.0,1.0],linewidths=2.5),plt.contour(y_predShow2_epi [0], colors=['r'], levels=[0.0,1.0],linewidths=2.5) , plt.axis('off')
  plt.subplot(175),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('Lac_loss'),plt.contour(y_predShow5 [0], colors=['r','g'], levels=[0.0,1.0,2.0],linewidths=2.5), plt.axis('off')
  plt.subplot(176),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('Lms_loss'),plt.contour(y_predShow4 [0], colors=['r','g'], levels=[0.0,1.0,2.0],linewidths=2.5), plt.axis('off')
  plt.subplot(177),plt.imshow(x_test_acdc[i,...,0].astype(np.int16),cmap='gray'),plt.title('Our proposed'),plt.contour(y_predShow3 [0], colors=['r','g'], levels=[0.0,1.0,2.0],linewidths=2.5), plt.axis('off')
  
 

Output hidden; open in https://colab.research.google.com to view.